# Advanced Microwave Scanning Radiometer (AMSR)

In [ ]:
import geopandas as gpd
import pandas as pd
import cartopy
import matplotlib.pyplot as plt
import json
import shapely
import os

import shared_functions as sf

# settings
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 150)

| Shortname | Description |
|--|--|
| AMSREL1A | AMSR-E/Aqua L1A Raw Observation Counts V003 |
| AMSR-L1A | AMSR/ADEOS-II L1A Raw Observation Counts V003 |
| GPM_1CGCOMW1AMSR2 | GPM AMSR-2 on GCOM-W1 Common Calibrated Brightness Temperature L1C 1.5 hours 10 km V07 (GPM_1CGCOMW1AMSR2) at GES DISC |
| GPM_1CAQUAAMSRE | GPM AMSR-E on AQUA Common Calibrated Brightness Temperatures L1C 1.5 hours 10.5 km V07 (GPM_1CAQUAAMSRE) at GES DISC |

## Functions

In [ ]:
def annual_summary(df, size_col = 'size_GB'):
    count_size = (
        (df.groupby(['year']).agg(
        count = ('id','count'),
        size = (size_col, 'sum')))
        .reset_index()
        #.pivot(index='year', values=['count','size'])
        .fillna(0)
    )
    count_size.loc["Total"] = count_size.sum()
    count_size['count'] = count_size['count'].astype(int)
    count_size[size_col] = (count_size['size']).round(2)
    return count_size

## AMSR Raw Observation Counts (1A)

In [ ]:
with open('metadata/AMSR-L1A_-50N_products.json') as f:
    data = json.load(f)
df = pd.DataFrame.from_dict(data, orient='index')
df = sf.preprocess_cmr_df(df, crs=3031, lon_first=True, aoi_filter=True)
print(df['time_start'].min(), df['time_start'].max())

In [ ]:
sf.plot_results_footprint_map(df.iloc[1:10])

In [ ]:
annual_summary(df)

## ASMR-E Raw Observation Counts (1A)

In [ ]:
with open('metadata/AMSREL1A_-50N_products.json') as f:
    data = json.load(f)
df = pd.DataFrame.from_dict(data, orient='index')
df = sf.preprocess_cmr_df(df, crs=3031, lon_first=True, aoi_filter=True)
print(df['time_start'].min(), df['time_start'].max())

In [ ]:
sf.plot_results_footprint_map(df.iloc[1:10])

In [ ]:
annual_summary(df, size_col='size_GB')

## GPM AMSR-E on AQUA Common Calibrated Brightness Temperatures L1C 1.5 hours

In [ ]:
with open('metadata/GPM_1CAQUAAMSRE_-50N_products.json') as f:
    data = json.load(f)
df = pd.DataFrame.from_dict(data, orient='index')
df = sf.preprocess_cmr_df(df, crs=3031, lon_first=True)
print(df['time_start'].min(), df['time_start'].max())

In [ ]:
title = 'GPM AMSR-E on AQUA Common Calibrated Brightness Temperatures L1C 1.5 hours'
sf.plot_results_footprint_map(df.iloc[0:10], title=title)
df.iloc[50:54].to_crs(4326).geometry.values[0]

In [ ]:
annual_summary(df, size_col='size_GB')

## GPM AMSR-2 on GCOM-W1 Common Calibrated Brightness Temperature L1C

In [ ]:
with open('metadata/GPM_1CGCOMW1AMSR2_-50N_products.json') as f:
    data = json.load(f)
df = pd.DataFrame.from_dict(data, orient='index')
df = sf.preprocess_cmr_df(df, crs=3031, lon_first=True)
print(df['time_start'].min(), df['time_start'].max())

In [ ]:
title = 'GPM AMSR-2 on GCOM-W1 Common Calibrated Brightness Temperature L1C'
sf.plot_results_footprint_map(df.iloc[0:10], title=title)

In [ ]:
annual_summary(df, size_col='size_GB')

## Level 2 data

In [ ]:
files = [x for x in os.listdir('metadata') if 'AMSR' == x.split('_')[0]]
files.remove('AMSR_L2_AE_Land_-50N_products.json')
files


In [ ]:
fname = files[0]
with open(f'metadata/{files[0]}') as f:
    data = json.load(f)
df = pd.DataFrame.from_dict(data, orient='index')
df = sf.preprocess_cmr_df(df, crs=3031, lon_first=True)
print(df['time_start'].min(), df['time_start'].max())

dfs = []

for fname in files:
    file_ = os.path.join('metadata',fname)
    with open(file_) as f:
        data = json.load(f)
    df = pd.DataFrame.from_dict(data, orient='index')
    df = sf.preprocess_cmr_df(df, crs=3031, lon_first=True)
    df['product'] = fname
    dfs.append(df)
    df = None

# same columns
cols = []
for d in dfs:
    cols += list(d)
cols = list(set(cols))
for d in dfs:
    for c in cols:
        if c not in list(d):
            d[c] = ''

df = pd.concat(dfs)

In [ ]:
title = fname
sf.plot_results_footprint_map(df.iloc[0:30], title=title)

In [ ]:
annual_summary(df, size_col='size_TB')

In [ ]:
s = df[df['year']==2022].groupby('product').agg(
        count = ('id','count'),
        size_TB = ('size_TB', 'sum'),
        size_MB = ('size_MB', 'sum'),
        start = ('time_start','min'),
        end = ('time_start','max')).reset_index().fillna(0)

s['start'] = s['start'].apply(lambda x : x.date())
s['end'] = s['end'].apply(lambda x : x.date())
s['mean_size_MB'] = s['size_MB']/s['count']
s

In [ ]:
def annual_summary(df, size_col = 'size_GB'):
    count_size = (
        (df.groupby(['year']).agg(
        count = ('id','count'),
        size = (size_col, 'sum')))
        .reset_index()
        #.pivot(index='year', values=['count','size'])
        .fillna(0)
    )
    count_size.loc["Total"] = count_size.sum()
    count_size['count'] = count_size['count'].astype(int)
    count_size[size_col] = (count_size['size']).round(2)
    return count_size

# Combined

In [ ]:
files = {
    'AMSR' : 'metadata/AMSR-L1A_-50N_products.json',
    #'' :'metadata/AMSREL1A_-50N_products.json',
    'AMSR-E' : 'metadata/GPM_1CAQUAAMSRE_-50N_products.json',
    'AMSR-2' : 'metadata/GPM_1CGCOMW1AMSR2_-50N_products.json',
}


dfs = []

for sat in files.keys():
    print(sat)
    file_ = files[sat]
    with open(file_) as f:
        data = json.load(f)
    df = pd.DataFrame.from_dict(data, orient='index')
    df = sf.preprocess_cmr_df(df, crs=3031, lon_first=True)
    df['satellite'] = sat
    dfs.append(df)

# same columns
cols = []
for d in dfs:
    cols += list(d)
cols = list(set(cols))
for d in dfs:
    for c in cols:
        if c not in list(d):
            d[c] = ''

df = pd.concat(dfs)

In [ ]:
title = f'AMSR level 1 - Weekly Products' 
sf.plot_timeseries_products(df.reset_index(), title=title, stack_col='satellite', date_col='time_start',count_freq='7D', plot_freq='1M')